cutline filling 
- what: transect are diagonal, and we want to create 64x64 patches for image models
- why: to train image mdoels 
- where: transect available (RNG) or (TLS)
- how: use bbox for the patches > buffer bbox n pixels 

In [1]:
import os 
from glob import glob

In [ ]:
cutline_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/cutline/"
bbox_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/bbox/"
outdir = exp_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment/"

In [37]:
pdem_fn= "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/pdem/pdem.vrt"
s1_fn=  "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/s1/s1.vrt"
s2_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/s2/s2.vrt"
esawc_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/esawc/esawc.vrt"
ldem_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/ldem/ldem.vrt"

Augmentation:
- bbox amz
- bbox mkg: buffer to increase the size (incrementally by radius of n pixel)

In [22]:
tiff_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_OverlapPatchesLiDAR/*.tif")

In [26]:
reproj_dir = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj"

In [48]:
### use magic tools to make into a python script 
import os 
import subprocess
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import box  

def clip_raster_by_vector_cutline(raster_fn, out_fn, bbox_fn):
    pass 

def clip_raster_by_vector_bbox(raster_fn, out_fn, bbox_fn):
    bbox = gpd.read_file(bbox_fn)
    with rasterio.open(raster_fn) as src:
        
        bbox = bbox.to_crs(src.crs)
        raster_bounds = box(*src.bounds)  # Convert bounds to a Shapely Polygon
        raster_gdf = gpd.GeoDataFrame(geometry=[raster_bounds], crs=src.crs)
        
        # Check if bbox overlaps with raster bounds
        if not bbox.intersects(raster_gdf).any():
            raise ValueError(f"No overlap between bbox and raster {raster_fn}")
        
        # Clip raster using bbox geometry
        out_image, out_transform = mask(src, bbox.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
    
    # Save clipped raster
    with rasterio.open(out_fn, "w", **out_meta) as dest:
        dest.write(out_image)


def reproject_files_seq(tiff_files, reproj_dir,epsgcode):
    #epsgcode = 4326 4979
    os.makedirs(reproj_dir, exist_ok=True)
    reprojected_files = []
    for tiff_file in tiff_files:
        # Construct the output file path for the reprojected file
        base_name = os.path.splitext(os.path.basename(tiff_file))[0]
        reproj_tiff = os.path.join(reproj_dir, f"{base_name}_reproj.tif")
        gdalreproject(fi=tiff_file, fo=reproj_tiff, epsgcode=epsgcode)
        reprojected_files.append(reproj_tiff)
    return reprojected_files

def reproject_files_par():
    pass 

def gdalreproject(fi, fo, epsgcode=4326):
    if not os.path.isfile(fo):
        # Use gdalwarp to reproject the file
        cmd_reproject = [
            "gdalwarp",
            "-t_srs", f"EPSG:{epsgcode}",  # Target projection (EPSG:4326)
            fi,                     # Input file
            fo                    # Output file
        ]
        
        try:
            subprocess.run(cmd_reproject, check=True)
            print(f"Reprojected {fi} -> \n{fo}")
        except subprocess.CalledProcessError as e:
            print(f"Error reprojecting {fi}: {e}")

def list2txt(txt, files):
    with open(txt, "w") as f:
        f.write("\n".join(files))


def gdalbuildvrt(vrt, txt, epsgcode):
    if not os.path.isfile(vrt):
        cmd_vrt = [
        "gdalbuildvrt",
        "-a_srs", f"EPSG:{epsgcode}",    # Assign the specified EPSG code
        "-input_file_list", txt,         # Input file list (text file)
        vrt                              # Output VRT file
        ]

        try:
            # Run the GDAL command to create the VRT
            subprocess.run(cmd_vrt, check=True)
            print(f"VRT file created successfully: {vrt}")
        except subprocess.CalledProcessError as e:
            print(f"Error creating VRT file: {e}")

In [ ]:
txt = 'transects.txt'
vrt = 'transects.vrt'
epsgcode = 4326
reprojected_files = reproject_files_seq(tiff_files, reproj_dir,epsgcode)
list2txt(txt, files=reprojected_files)
gdalbuildvrt(vrt, txt, epsgcode)

In [11]:
bbox_files = glob(f"{bbox_dir}/*DTM_NP*.gpkg")
bbox_fn = bbox_files[0]
outdir = exp_dir

In [ ]:

# Save clipped rasters into outdir
clip_raster_by_vector_bbox(vrt, f"{outdir}/clipped_ldem.tif", bbox_fn)
clip_raster_by_vector_bbox(s1_fn, f"{outdir}/clipped_s1.tif", bbox_fn)

In [ ]:
xpath = f"{outdir}/clipped_s1.tif" #2bands
ypath = f"{outdir}/clipped_ldem.tif" #1band
zpath = f"{outdir}/clipped_ldem_pred.tif" #1band

# load xpath and ypath
# ypath contains nulls, remove them and filter the train data 
# fit RF regression 
# make prediction 
# get RMSE, R2 and print
# save prediction zpath 1 
# save prediction 2 to zpath2 where you only predict where ypath is nulls, and merge the prediction and the original data

In [ ]:
import os
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from rasterio.warp import reproject, Resampling 

# fix the code to predict dem - target 

# File paths
xpath = f"{outdir}/clipped_s1.tif"  # 2 bands (X) S1 features
ypath = f"{outdir}/clipped_ldem.tif"  # 1 band (y) dem target
zpath1 = f"{outdir}/clipped_ldem_pred.tif"  # Prediction output (1)
zpath2 = f"{outdir}/clipped_ldem_pred_merged.tif"  # Prediction merged with original data

# Step 1: Load X and y from TIFF files
with rasterio.open(xpath) as src_x, rasterio.open(ypath) as src_y:
    # Reproject ypath to match xpath's CRS if they differ
    if src_x.crs != src_y.crs:
        print("Reprojecting ypath to match xpath's CRS...")
        # Create an empty array for the reprojected y_data
        y_reprojected = np.empty(src_x.shape, dtype=np.float32)
        
        # Reproject y_data to match xpath's CRS
        reproject(
            source=rasterio.band(src_y, 1),
            destination=y_reprojected,
            src_transform=src_y.transform,
            src_crs=src_y.crs,
            dst_transform=src_x.transform,
            dst_crs=src_x.crs,
            resampling=Resampling.nearest
        )
        
        # Replace NoData values with np.nan
        nodata_value = src_y.nodata
        if nodata_value is not None:
            y_reprojected[y_reprojected == nodata_value] = np.nan
        
        # Use the reprojected y_data
        y_data = y_reprojected
        y_profile = src_x.profile  # Use the profile of xpath for consistency
    else:
        # If CRS is the same, read y_data directly
        y_data = src_y.read(1).astype(np.float32)
        nodata_value = src_y.nodata
        if nodata_value is not None:
            y_data[y_data == nodata_value] = np.nan
        y_profile = src_x.profile  # Use the profile of xpath for consistency
    
    # Read all bands of X
    x_data = src_x.read()  # Shape: (bands, height, width)
    x_profile = src_x.profile

# Flatten X and y for training
x_flat = x_data.reshape(x_data.shape[0], -1).T  # Shape: (pixels, bands)
y_flat = y_data.flatten()  # Shape: (pixels,)

# Filter out null values in y
valid_mask = ~np.isnan(y_flat)
if valid_mask.sum() == 0:
    raise ValueError("No valid pixels found in y_data after filtering NaNs.")

x_train = x_flat[valid_mask]
y_train = y_flat[valid_mask]

# Step 2: Train Random Forest Regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Step 3: Make predictions on the entire dataset
y_pred_flat = rf_model.predict(x_flat)

# Reshape predictions back to the original raster shape
y_pred = y_pred_flat.reshape(y_data.shape)

# Step 4: Evaluate model performance (RMSE, R2)
rmse = np.sqrt(mean_squared_error(y_train, rf_model.predict(x_train)))
r2 = r2_score(y_train, rf_model.predict(x_train))
print(f"Model Performance - RMSE: {rmse:.4f}, R2: {r2:.4f}")

# Step 5: Save prediction to zpath1
with rasterio.open(zpath1, 'w', **y_profile) as dst:
    dst.write(y_pred.astype(rasterio.float32), 1)

# Step 6: Predict only where y is null and merge with original data
y_pred_nulls = np.where(np.isnan(y_data), y_pred, y_data)  # Replace nulls with predictions

# Save the merged prediction to zpath2
with rasterio.open(zpath2, 'w', **y_profile) as dst:
    dst.write(y_pred_nulls.astype(rasterio.float32), 1)

print(f"Predictions saved to {zpath1} and {zpath2}")

Reprojecting ypath to match xpath's CRS...
Model Performance - RMSE: 1.1251, R2: 0.8499
Predictions saved to /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment//clipped_ldem_pred.tif and /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment//clipped_ldem_pred_merged.tif


In [39]:
# use the residual to improve the ouptput 
# synthetic stuff becomes hand here to increase sample size 
outdir = exp_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment/"

In [40]:
import os
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from rasterio.warp import reproject, Resampling

# File paths

xpath = f"{outdir}/clipped_s1.tif"  # 2 bands (X) S1 features
ypath = f"{outdir}/clipped_ldem.tif"  # 1 band (y) DEM target
zpath1 = f"{outdir}/clipped_ldem_pred.tif"  # Prediction output (1)
zpath2 = f"{outdir}/clipped_ldem_pred_merged.tif"  # Prediction merged with original data

# Step 1: Load X and y from TIFF files
with rasterio.open(xpath) as src_x, rasterio.open(ypath) as src_y:
    # Reproject ypath to match xpath's CRS if they differ
    if src_x.crs != src_y.crs:
        print("Reprojecting ypath to match xpath's CRS...")
        # Create an empty array for the reprojected y_data
        y_reprojected = np.empty(src_x.shape, dtype=np.float32)
        
        # Reproject y_data to match xpath's CRS
        reproject(
            source=rasterio.band(src_y, 1),
            destination=y_reprojected,
            src_transform=src_y.transform,
            src_crs=src_y.crs,
            dst_transform=src_x.transform,
            dst_crs=src_x.crs,
            resampling=Resampling.nearest
        )
        
        # Replace NoData values with np.nan
        nodata_value = src_y.nodata
        if nodata_value is not None:
            y_reprojected[y_reprojected == nodata_value] = np.nan
        
        # Use the reprojected y_data
        y_data = y_reprojected
        y_profile = src_x.profile  # Use the profile of xpath for consistency
    else:
        # If CRS is the same, read y_data directly
        y_data = src_y.read(1).astype(np.float32)
        nodata_value = src_y.nodata
        if nodata_value is not None:
            y_data[y_data == nodata_value] = np.nan
        y_profile = src_x.profile  # Use the profile of xpath for consistency
    
    # Read all bands of X
    x_data = src_x.read()  # Shape: (bands, height, width)
    x_profile = src_x.profile

# Flatten X and y for training
x_flat = x_data.reshape(x_data.shape[0], -1).T  # Shape: (pixels, bands)
y_flat = y_data.flatten()  # Shape: (pixels,)

# Filter out null values in y
valid_mask = ~np.isnan(y_flat)
if valid_mask.sum() == 0:
    raise ValueError("No valid pixels found in y_data after filtering NaNs.")

x_train = x_flat[valid_mask]
y_train = y_flat[valid_mask]

# Step 2: Train Random Forest Regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(x_train, y_train)

# Step 3: Make predictions on the entire dataset
y_pred_flat = rf_model.predict(x_flat)

# Reshape predictions back to the original raster shape
y_pred = y_pred_flat.reshape(y_data.shape)

# Step 4: Evaluate model performance (RMSE, R2)
rmse = np.sqrt(mean_squared_error(y_train, rf_model.predict(x_train)))
r2 = r2_score(y_train, rf_model.predict(x_train))
print(f"Model Performance - RMSE: {rmse:.4f}, R2: {r2:.4f}")

# Step 5: Save prediction to zpath1
y_profile.update(dtype=rasterio.float32, count=1, nodata=np.nan)
with rasterio.open(zpath1, 'w', **y_profile) as dst:
    dst.write(y_pred.astype(rasterio.float32), 1)

# Step 6: Predict only where y is null and merge with original data
y_pred_nulls = np.where(np.isnan(y_data), y_pred, y_data)  # Replace nulls with predictions

# Save the merged prediction to zpath2
with rasterio.open(zpath2, 'w', **y_profile) as dst:
    dst.write(y_pred_nulls.astype(rasterio.float32), 1)

print(f"Predictions saved to {zpath1} and {zpath2}")

Reprojecting ypath to match xpath's CRS...
Model Performance - RMSE: 1.1251, R2: 0.8499
Predictions saved to /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment//clipped_ldem_pred.tif and /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment//clipped_ldem_pred_merged.tif


In [ ]:
# outside of lidar just predicting noise 
#  options [better model trained longer with/wihtout synth @write results ]:
#a: use dsm instead instead of S1
#b: use dsm to dx, but find a way to better extrapolate (beyond noise): use zdiff
#c: pinn constain the model to min,max of the lidar so long no more than dsm max

#> once we have a image filled bbox, we can use cn
### i am doing this so to have rectange or triangle images that i can use for ML